In [32]:
""" Class 9. Model Evaluation and Bias Variance Tradeoff
"""

import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib notebook
plt.style.use('deeplearning.mplstyle')

In [33]:
housing_dataset = pd.read_csv('housing.csv')
housing_dataset.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [34]:
df=housing_dataset[['area','price']]
df.head()

,area,price
0,7420,13300000
1,8960,12250000
2,9960,12250000
3,7500,12215000
4,7420,11410000


In [35]:
df=(df-df.mean())/df.std()
df.head()

,area,price
0,1.045766,4.562174
1,1.755397,4.000809
2,2.216196,4.000809
3,1.082630,3.982096
4,1.045766,3.551716


In [37]:
seed = 42
np.random.seed(seed)

def split_dataset(dataset, training_ratio=0.6, val_ratio=0.2):
    no_of_examples = len(dataset)
    random_indices = np.arange(no_of_examples)
    
    train_size = int(training_ratio * no_of_examples)
    val_size = int(val_ratio * no_of_examples)
    
    training_indices = random_indices[:train_size]
    val_indices = random_indices[train_size:val_size + train_size]
    test_indices = random_indices[val_size + train_size:]
    
    train_ds = dataset.iloc[training_indices]
    val_ds = dataset.iloc[val_indices]
    test_ds = dataset.iloc[test_indices]
    
    train_X = (train_ds.iloc[:, :-1])
    train_y = train_ds.iloc[:, -1]
    
    val_X = val_ds.iloc[:, :-1]
    val_y = val_ds.iloc[:, -1]
    
    test_X = test_ds.iloc[:, :-1]
    test_y = test_ds.iloc[:, -1]
    
    return train_X, train_y, val_X, val_y, test_X, test_y
    
train_X, train_y, val_X, val_y, test_X, test_y = split_dataset(df)

In [ ]:
print(train_X.shape)

# **Polynomial Regression**

In [38]:
def get_house_price(x,w1,w2,b):
    y_pred=w1*x**2+w2*x+b
    return y_pred
def cost_function(x,y_true,w1,w2,b):
    y_pred=get_house_price(x,w1,w2,b)
    mse=np.mean((y_true-y_pred)**2)
    return mse

In [39]:
w1=np.random.randint(100,200)
w2=np.random.randint(100,200)
b=np.random.randint(100,200)

In [52]:
def cost_function(x, y_true, w1, w2, b):
    lamda = 0.5
    y_pred = get_house_price(x, w1, w2, b)
    mse = np.mean((y_pred - y_true) ** 2) # Depends on the Task
    # regularization_term=w1**2+w2**2+b
    regularizer_term = np.abs(w1) + np.abs(w2) + np.abs(b)
    return mse+lamda*regularizer_term 

In [41]:
mse = cost_function(train_X.iloc[:,0], train_y, w1, w2, b)
print(f"MSE is {mse:0.2f}  (Parameters not learned yet.)")

MSE is 436193.90  (Parameters not learned yet.)


In [42]:
""" ML Workflow | Step 8: Optimizers selection: Adam """

def compute_gradient(X, y_true, w1, w2, b):
    delta = 1e-9
    
    cost_1 = cost_function(X, y_true, w1, w2, b)
    cost_2 = cost_function(X, y_true, w1 + delta, w2, b)
    cost_3 = cost_function(X, y_true, w1, w2 + delta, b)
    cost_4 = cost_function(X, y_true, w1, w2, b + delta)
    
    dw1 = (cost_2 - cost_1) / delta
    dw2 = (cost_3 - cost_1) / delta
    db = (cost_4 - cost_1) / delta
    return dw1, dw2, db

In [53]:
""" Train the model """

def train_model(train_X, train_y, val_X, val_y, epochs=2000, lr=0.001):
    """ Initialize the parameters """
    w1 = 0
    w2 = 0
    b = 0
    
    for epoch in range(epochs):
        """ compute logits """
        loss = cost_function(train_X.iloc[:,0], train_y, w1, w2, b)
        val_loss = cost_function(val_X.iloc[:,0], val_y, w1, w2, b)
        
        """ compute gradient """
        dw1, dw2, db = compute_gradient(train_X.iloc[:,0], train_y, w1, w2, b)
        
        """ update parameters """
        w1 = w1 - lr * dw1
        w2 = w2 - lr * dw2
        b = b - lr * db
        
        if epoch % 100 == 0:
            print(f"Epoch {epoch} / {epochs} loss: {loss:.4f} val_loss: {val_loss:.4f}")
            
    return w1, w2, b
    
w1, w2, b = train_model(train_X, train_y, val_X, val_y)
train_loss = cost_function(train_X.iloc[:,0], train_y, w1, w2, b)
print(f"w1: {w1}, w2: {w2}, b: {b}, loss={train_loss:.4f}")

Epoch 0 / 2000 loss: 1.1172 val_loss: 0.3961
Epoch 100 / 2000 loss: 1.0132 val_loss: 0.6047
Epoch 200 / 2000 loss: 1.0011 val_loss: 0.6760
Epoch 300 / 2000 loss: 0.9938 val_loss: 0.7175
Epoch 400 / 2000 loss: 0.9885 val_loss: 0.7497
Epoch 500 / 2000 loss: 0.9846 val_loss: 0.7764
Epoch 600 / 2000 loss: 0.9818 val_loss: 0.7990
Epoch 700 / 2000 loss: 0.9797 val_loss: 0.8179
Epoch 800 / 2000 loss: 0.9781 val_loss: 0.8337
Epoch 900 / 2000 loss: 0.9770 val_loss: 0.8470
Epoch 1000 / 2000 loss: 0.9761 val_loss: 0.8581
Epoch 1100 / 2000 loss: 0.9755 val_loss: 0.8672
Epoch 1200 / 2000 loss: 0.9750 val_loss: 0.8748
Epoch 1300 / 2000 loss: 0.9746 val_loss: 0.8811
Epoch 1400 / 2000 loss: 0.9743 val_loss: 0.8863
Epoch 1500 / 2000 loss: 0.9741 val_loss: 0.8905
Epoch 1600 / 2000 loss: 0.9739 val_loss: 0.8939
Epoch 1700 / 2000 loss: 0.9738 val_loss: 0.8967
Epoch 1800 / 2000 loss: 0.9737 val_loss: 0.8990
Epoch 1900 / 2000 loss: 0.9736 val_loss: 0.9008
w1: 0.033619919848071333, w2: 0.12356727518980648, b